In [1]:
'''
TODO
- cell padding
- width jó legyen
- ne kelljen görgetni

'''

'\nTODO\n- cell padding\n- width jó legyen\n- ne kelljen görgetni\n\n'

# Packages

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import pandas as pd
import pprint

DATAPATH = 'C:/Users/HP/Documents/02_CODING/KURZUS_Network/DATA/Kurzushetvege/'

# External stylesheet

In [3]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

# Call the app

In [4]:
# In a .py file, use this:
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Now, in a Jupyter notebook environment, we use this:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Formatting presets

In [5]:
colors = {
    'bg' : '#a8dadc'
}

In [6]:
udvozlo_szoveg = '''
De jó, hogy a dashboardunkra tévedtél! Próbáld ki, mielőtt _migráncsoznál_ egy jót!
A grafikonokon a World Bank migrációs adatbázisának adatai láthatók.
Erről ezeket érdemes tudni:
- bla
- bliblu
- bluble

[Adatok leírása](https://datacatalog.worldbank.org/dataset/global-bilateral-migration-database)

[Adatok forrása](https://www.worldbank.org/en/topic/migrationremittancesdiasporaissues/brief/migration-remittances-data)

Az alábbi gombok megnyomásával megnézhetsz néhány érdekes sztorit!


'''

# Getting the data

In [7]:
koldf = pd.DataFrame({
    'koli' : ['rajk', 'szisz', 'evk'],
    'iq' : [150, 130, 110],
    'arc' : [8, 6, 10],
    'color' : ['green', 'black', 'red']
    })

all_koli = koldf.koli.unique()

In [8]:
edges = pd.read_csv(DATAPATH + 'edge_list_final.csv')
nodes = pd.read_csv(DATAPATH + 'attributes.csv')
all_from = edges.Origin.unique()
all_to = edges.Destination.unique()
all_year = edges.Year.unique()

In [9]:
edges.sample(5)

,Unnamed: 0,Origin,Destination,Year,Stock,Flow,Origin_latitude,Origin_longitude,Destination_latitude,Destination_longitude
19459,19459,United States,West Bank and Gaza,2000,41175.0,14538,37.090240,-95.712891,NaN,NaN
4503,4503,Czech Republic,India,1990,2711.0,2621,49.817492,15.472962,20.593684,78.962880
7895,7895,India,Israel,1990,16249.0,-4431,20.593684,78.962880,31.046051,34.851612
16264,16264,Sri Lanka,Canada,2000,89335.0,78537,7.873054,80.771797,56.130366,-106.346771
13797,13797,Poland,Argentina,1960,113735.0,113735,51.919438,19.145136,-38.416097,-63.616672


In [10]:
nodes.sample(5)

,Origin,Year,Origin_latitude,Origin_longitude,Stock
624,Honduras,2013,15.199999,-86.241905,6.478760e+05
253,Canada,2017,56.130366,-106.346771,1.228500e+06
1282,South Africa,1960,-30.559482,22.937506,1.721540e+05
1189,Rwanda,2010,-1.940278,29.873888,2.310928e+05
889,Malaysia,2013,4.210484,101.975766,1.666829e+06


In [11]:
len(all_to)

211

# Defining functions and figures

In [12]:
# Példa
fig1 = px.bar(koldf, x='koli', y='iq', color = 'koli', color_discrete_sequence = koldf['color'])
fig2 = px.bar(koldf, x='koli', y='arc', color = 'koli', color_discrete_sequence = koldf['color'])

In [13]:
# Mendöl térkép készítője
from mapping import map_creation
# Gyenes gráf készítője
from gyenes_vizu import generate_data, graf_vizu

# App layout

In [14]:
app.layout = html.Div(children=[
    html.H1(children='Migráció app', style = {'backgroundColor' : colors['bg']}),
    html.H4(children='Rajk Network Science kurzushétvége, 2021.04.16-18.', style = {'backgroundColor' : colors['bg']}),
    
    dbc.Row(
            [
            dbc.Col(
                [
                    html.Div('Migráció a világban', style = {'textAlign' : 'left', 'fontSize' : 20, 'fontWeight' : 'bold'}),
                    html.Div('Egy adott év kiválasztásával megtekinthető az évben országok közötti áramlás', style = {'textAlign' : 'left'}),
                    #html.Img(src = 'https://cdn.vox-cdn.com/thumbor/2-7zdMpMpfbDDqIn2GsRVNN-au8=/51x0:977x617/1400x1050/filters:focal(51x0:977x617):format(png)/cdn.vox-cdn.com/uploads/chorus_image/image/50017015/Screen_Shot_2016-07-04_at_12.37.15_PM.0.0.png',
                    #         height = 300, width = 420),
                    
                    # Mendöl térképe
                    dcc.Graph(id = 'terkep_viz'),
                    
    
                    dcc.Slider(
                        id = 'year_slider_1',
                        min = all_year.min(),
                        max = all_year.max(),
                        value = all_year.max(),
                        marks = {str(year): str(year) for year in all_year},
                        step = None)
                    
                    
                ], style = {'backgroundColor' : colors['bg']}, width = {'size' : 6}
                ),
            
            dbc.Col(
                [
                    html.Div('Migrációs hálózat', style = {'textAlign' : 'left', 'fontSize' : 20, 'fontWeight' : 'bold'}),
                    html.Div('Országok és év kiválasztásával megtekinthető az adott országok adott éves migrációs hálózata', style = {'textAlign' : 'left'}),
                    #html.Img(src = 'https://brsl.net.bd/wp-content/uploads/2020/04/1_rmq7bd3GFjcwfXtkrBQaPQ.png',
                    #         height = 300, width = 600),
                    
                    # Gyenes gráfja
                    html.Iframe(id = 'graf_viz', width = '100%', height = '75%'),
                    
                    
                    dcc.Slider(
                        id = 'year_slider_2',
                        min = all_year.min(),
                        max = all_year.max(),
                        value = all_year.max(),
                        marks = {str(year): str(year) for year in all_year},
                        step = None),
                    dcc.Dropdown(
                        id = 'orszag_dropdown_2', # this number means that this is the 2nd cell on the dashboard (not that this is the 2nd dropdown filter)
                        options = [{'label': i, 'value': i} for i in all_from],
                        value = ['Germany', 'Greece', 'Austria', 'Hungary', 'Turkey'],
                        multi=True
                        )
                    
                    
                    
                ], style = {'backgroundColor' : colors['bg']}, width = {'size' : 6}),
            
            ]), # Row 1 vége
    
    html.Br(),
            
    dbc.Row(
            [
            
            dbc.Col(
                [
                    html.Div('Adott ország vizsgálata', style = {'textAlign' : 'left', 'fontSize' : 20, 'fontWeight' : 'bold'}),
                    html.Div('Egy adott ország esetében megtekinthetők a leggyakoribb küldő vagy fogadó országok', style = {'textAlign' : 'left'}),
                    html.Img(src = 'https://www.amcharts.com/wp-content/uploads/2018/01/horizontal-bar-chart.png',
                             height = 300, width = 384),
                    dcc.Dropdown(
                        id = 'orszag_dropdown_3', # this number means that this is the 3rd cell on the dashboard (not that this is the 3rd dropdown filter)
                        options = [{'label': i, 'value': i} for i in all_from],
                        value = 'Germany'
                        ),
                    dcc.RadioItems(
                        id = 'kibe_radio_3',
                        options = [{'label': 'Beáramlás', 'value' : 'be'},
                                   {'label': 'Kiáramlás', 'value' : 'ki'}], # Itt még át kell írni a value-kat!
                        value = 'be',
                        labelStyle = {'display': 'inline-block'}
                    ),
                ],
                style = {'backgroundColor' : colors['bg']}, width = {'size' : 6}),
            
            dbc.Col(
                [
                    html.Div('Útmutató', style = {'textAlign' : 'left', 'fontSize' : 20, 'fontWeight' : 'bold'}),
                    #dcc.Markdown(udvozlo_szoveg),
                    #html.Iframe(probahtml)
                ],
                style = {'backgroundColor' : colors['bg']}, width = {'size' : 6}),
                
                #[
                #    html.Div("Jobb alsó", style = {'textAlign' : 'center', 'fontSize' : 14}),
                #    html.Div("A szakkolisok IQ-ja", style = {'textAlign' : 'center', 'fontSize' : 20}),
                #    html.Br(),
                #    dcc.Graph(
                #        id = 'iq_graf'
                #        ),
                #    html.Label('Koli szűrő'),
                #    dcc.Dropdown(
                #        id = 'koli_dropdown1',
                #        options = [{'label': i, 'value': i} for i in all_koli],
                #        value = ['rajk', 'szisz'],
                #        multi=True
                #        )
                #], style = {'backgroundColor' : colors['bg']}, width = {'size' : 6})],
                
            ]) # Row 2 vége
    ]) # Layout vége

# Callbacks

In [15]:
#@app.callback(
#    Output('iq_graf', 'figure'),
#    Input('koli_dropdown1', 'value'))
#def update_figure(kolik):
#    filtered_df = koldf[koldf.koli.isin(kolik)]
#
#    fig = px.bar(filtered_df, x='koli', y='iq', color = 'koli', color_discrete_sequence = list(filtered_df['color']))
#
#    fig.update_layout(transition_duration=500)
#
#    return fig

# TÉRKÉP (bal felső)
@app.callback(
    Output('terkep_viz', 'figure'),
    Input('year_slider_1', 'value'))
def update_map(year):
    szurt_nodes = nodes[nodes.Year == year]
    szurt_edges = edges[edges.Year == year]
    fig = map_creation(edgelist_df = szurt_edges, node_df = szurt_nodes, edgeweight_multiplier = 0.02)
    fig.update_layout(transition_duration = 500)
    return fig

# GRÁF (jobb felső)
@app.callback(
    Output('graf_viz', 'srcDoc'),
    Input('year_slider_2', 'value'),
    Input('orszag_dropdown_2', 'value'))
def update_graf(year, orszagok):
    szurt_nodes = nodes[(nodes.Year == year) & (nodes.Origin.isin(orszagok))]
    szurt_edges = edges[(edges.Year == year) & (edges.Origin.isin(orszagok)) & (edges.Destination.isin(orszagok))]
    #generalt_nodes, generalt_edges = generate_data(node_features = szurt_nodes, edge_features = szurt_edges)
    #fig_html = graf_vizu(node_features = generalt_nodes, edge_features = generalt_edges).html
    fig = graf_vizu(node_features = szurt_nodes, edge_features = szurt_edges)
    fig.save_graph('gyenes_graf.html')
    return fig.html # open('gyenes_graf.html', 'r').read()
    
    
    





# Run the app!

In [16]:
if __name__ == '__main__':
    app.run_server(debug=True, mode = 'external')

Dash app running on http://127.0.0.1:8050/


C:\Users\HP\Documents\02_CODING\KURZUS_Network\network_2021_spring\kurzushetvege\mapping.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\Documents\02_CODING\KURZUS_Network\network_2021_spring\kurzushetvege\mapping.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Próba

In [17]:
orszagok = ['Germany', 'Greece', 'Austria', 'Hungary', 'Turkey']

In [18]:
szurt_nodes = nodes[(nodes.Year == 2000) & (nodes.Origin.isin(orszagok))]
szurt_edges = edges[(edges.Year == 2000) & (edges.Origin.isin(orszagok)) & (edges.Destination.isin(orszagok))]

In [19]:
szurt_nodes.head(2)

,Origin,Year,Origin_latitude,Origin_longitude,Stock
82,Austria,2000,47.516231,14.550072,506555.0
528,Germany,2000,51.165691,10.451526,3549124.0


In [20]:
szurt_edges.head(4)

,Unnamed: 0,Origin,Destination,Year,Stock,Flow,Origin_latitude,Origin_longitude,Destination_latitude,Destination_longitude
996,996,Austria,Germany,2000,189096.0,37995,47.516231,14.550072,51.165691,10.451526
1093,1093,Austria,Turkey,2000,14309.0,7411,47.516231,14.550072,38.963745,35.243322
6363,6363,Germany,Austria,2000,101883.0,13073,51.165691,10.451526,47.516231,14.550072
6478,6478,Germany,Greece,2000,11349.0,-9744,51.165691,10.451526,39.074208,21.824312


In [21]:
fig = graf_vizu(node_features = szurt_nodes, edge_features = szurt_edges)

In [22]:
fig.show("mig.html")

In [23]:
gnodes,gedges=generate_data(szurt_nodes, szurt_edges)

In [24]:
gnodes

,stock_migration,display_nodesize
Origin,,
Austria,506555.0,93.981763
Germany,3549124.0,107.910995
Greece,1020252.0,98.991390
Hungary,409532.0,92.460513
Turkey,2977856.0,106.655338


In [25]:
gedges

,origin,destination,stock_migration,migration,display_edgesize
0,Germany,Austria,101883.0,13073,0.023833
1,Greece,Austria,2655.0,1096,0.001998
2,Hungary,Austria,17955.0,1598,0.002913
3,Turkey,Austria,179638.0,-4187,-0.007633
4,Austria,Germany,189096.0,37995,0.069269
5,Greece,Germany,367554.0,109477,0.199588
6,Hungary,Germany,71830.0,10745,0.019589
7,Turkey,Germany,2008979.0,548514,1.000000
8,Germany,Greece,11349.0,-9744,-0.017764
9,Turkey,Greece,7603.0,-20143,-0.036723
